# 6.0 Introduction


What is RAG?

RAG 는 Retrieval Augmented Generation (검색 증강 생성) 을 뜻한다.  

일반적으로 모델들은 개인적인 데이터들(개인 DB 나 문서)에는 때론 접근 할 수 없다.   
그래서 우리는 RAG 를 사용한다. 이게 무슨 말이냐면, 우리가 질문했을 때 우리는 2 가지 일을 하게 된다.  

예를 들어보자, 

'foo 가 뭐지?' 란 질문을 하면 우리는 그 질문(question)을 prompt 에 전달한다.  
또한 동시에 우리의 질문과 관련이 있는 문서들도 준비하게 된다.   
그래서 'foo'와 관련된 많은 문서들을 가져 올 것이다.  
그리고 그 문서들은 context 로써 우리의 질문(question)과 함께 묶어서 큰 language model 에 보낸다.  

그럼 이제 모델은 기존에 학습된 수 많은 data와 함께 우리가 model 이 더 나은 답변을 하도록 도와주기 위해 추가로 전송한 data 까지 갖게 된다. 

이게 바로 Retrieval Augmented Generation(RAG)이다.

개인으로부터 제공된 데이터를 사용하거나 탐색함으로써 우리가 Language Model 의 능력을 더 확장시키는 것.  
우리의 질문을 우리 개인의 문서와 함께 prompt 에 합친다.  

그 중 Stuff documents 는 모든 문서를 prompt 에 넣고 model 에 전달하는 방식이다.







# 6.1 Data loaders and Splitters

RAG 의 첫 번째 단계에 대해 알아보자  - Retrieval

Retrieval 는 랭체인의 모듈이다.

데이터를 Retrieval 하는 과정은 다음과 같다.

 어떤 소스에서 데이터를 읽어오고(Load) 분할(Transform)하면서 변환하고, Embed 하고, Store 한다.  
 <Embed - 텍스트에서 컴퓨터가 이해할 수 있는 숫자로 변환  ><br>
 <Store - 저장.  >

분할하는 이유는 embed 때문이다.

데이터를 읽어오는 loader 에 대해 알아보자.

Document loader 는 사용하기 쉽다. 



In [2]:
from langchain.document_loaders import UnstructuredFileLoader

data_path = "./files/chapter_one.txt"
loader = UnstructuredFileLoader(data_path)
loader.load()

[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a 

다음은 문서를 분할해보자.

문서를 분할해야하는 이유는 loader.load 의 리턴값을 보면 Document 로 이뤄진 리스트인데 위 경우에는 전체 챕터가 하나의 문서에 들어가 있다.

확인해보자.

In [3]:
len(loader.load())

1

문서가 하나의 큰 덩어리 이기 때문에 나누는 작업이 필요하다.

만약 네가 문서를 임베드, 저장하거나 언어 모델에 주고 싶다면 질문에 답해야할 때 필요한 '파일의 부분들'만을 전달할 수 있다.

만약 위 문서 중 특정 문자열인 'The Ministry of Peace' 부분을 찾고자 한다고 치자.

그럼 "Ministry of Peace" 가 어디 있을까 라고 질문을 던지면 

이 문자열을 가지고 있는 문서만 언어 모델에 주면 된다.

그러기 위해서 챕터 하나를 분할해야한다.






그럼 이제 분할하는 방법에 대해 알아보자.

분할하는 방법도 여러가지이다. 

이번에는 chatGPT 3, 4, OpenAI 와 잘 작동하는 걸 사용해본다. 나중에.

아주 일반적인 Text Splitter를 사용해보자.



In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter()

RecursiveCharacterTextSplitter 는 문장 끝이나 문단의 끝부분마다 끊어준다.

문장 중간을 끊어주진 않는다.

splitter 를 불러오면 두 가지 옵션이 있다.

첫 번째 옵션..

In [5]:
loader = UnstructuredFileLoader(data_path)
docs = loader.load()
splitter.split_documents(docs)

[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a 

두 번째 옵션..

loader.load_and_split()

In [6]:
loader = UnstructuredFileLoader(data_path)
loader.load_and_split(text_splitter=splitter)

[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a 

얼마나 큰 덩어리로 나눌지도 결정할 수 있다.

In [7]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
)
loader = UnstructuredFileLoader(data_path)
loader.load_and_split(text_splitter=splitter)

[Document(page_content='Part 1, Chapter 1\n\nPart One', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='It depicted simply an enormous face, more than a metre wide: the face of a man of abou

다른 splitter 도 있다.

CharacterTextSplitter

얘도 작동 방식이 비슷하다.

chunk size 가 있고, chunk Overlap 이 있다.

그리고 separator 가 있어서 특정 문자열을 찾은 다음 거기부터 끊는다.

In [8]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader(data_path)
loader.load_and_split(text_splitter=splitter)

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

[Document(page_content='Part 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The fl

In [9]:
len(loader.load_and_split(text_splitter=splitter))

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

45

# 6.2 Tiktoken

기본적으로, 이 모든 splitter 들은 텍스트의 length를 계산해서 한 덩어리의(chunk) 크기를 알아낸다.

그 작업에 파이썬 표준 라입르ㅓ리가 지우너하는 표준 len 함수를 사용한다.

여기서 너만의 len 함수를 명시적으로 설정할 수 있다.

예를 들어, 'ABCD' 를 입력하면, len 이, 문자의 개수인 '4'를 반환한다.

하지만 LLM 들은 Token 을 같은 방법으로 세어주지 않는다.

openAI 의 tokenizer 에 들어가면 model 의 관점에서 우리가 몇 개의 token을 사용하는지 볼 수 있다.

우리가 할 건 이 tokenizer(토큰화) 함수를 splitter 에 사용해 보는 것이다.

그러기 위해서 from_tiktoken_encoder 를 사용한다.

또 이것을 사용하는 이유는 모델이 텍스트를 세는 방법과 우리가 텍스트를 세는 방법을 일치하기 위해서이다.



In [10]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 6.3 Vectors

이번에는 Embedding 에 대해 알아보자.

임베딩의 의미는 사람이 읽는 텍스트를 컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업이다.

먼저 벡터에 대해 알아보자.

벡터, 정확히 vectorization(벡터화) 작업을 하게 될 건데, 우리가 만든 문서마다 각각의 벡터를 만들어준다.

즉, 나중에는 split 처리한 문서마다 embed 작업을 해줄 것이다.

우리가 이번에 사용해볼 embed 모델은 openai 의 모델이고 이건 최소 1000차원을 갖는 벡터를 제공한다.


3차원을 이용해서 벡터에 대해 알아보자.

```
            Masculinity(남성성) | Femininity(여성성) | Royalty(왕족성)
king    |       0.9             |       0.1          |      1.0
queen   |       0.1             |       0.9         |       1.0
man     |       0.9             |       0.1         |       0.0
```

위에서 우리는 점수를 매겼다.

각 단어가 해당 특성을 얼마나 반영하는지 평가한 것이다.

이게 우리의 3D 벡터의 차원들이다.

단어를 이런 차원들로 표현할 때의 멋진점은 

다른 단어를 얻기 위해 가진 단어를 가지고 연산을 할 수 있다는 점이다.

만약 king - man 에 대한 계산을 해본다면,

```
            Masculinity(남성성) | Femininity(여성성) | Royalty(왕족성)
king    |       0.9             |       0.1          |      1.0
man     |       0.9             |       0.1         |       0.0
             (0.9 - 0.9)             (0.1 - 0.1)    |    (1.0 - 0.0)
???     |       0.0             |       0.0         |       1.0
```
결과 단어는 0% 의 Masculinity 와 Feminity , 그리고 100% 의 royalty 를 갖는다.

아마 단어는 Royal 일 것이다.

성별은 몰라도 왕족이 확실함(100% Royalty)은 알 수 있다.

다음은 woman 이란 단어로 연산을 해보자.

```
            Masculinity(남성성) | Femininity(여성성) | Royalty(왕족성)
king    |       0.9             |       0.1          |      1.0
man     |       0.9             |       0.1         |       0.0
royal   |       0.0             |       0.0         |       1.0
woman   |       0.1             |       0.9         |       0.0
```

이번에는 royal + woman 연산을 해보자.
```
royal   |       0.0             |       0.0         |       1.0
woman   |       0.1             |       0.9         |       0.0
            (0.0 + 0.1)             (0.0 + 0.9)          (1.0 + 0.0)
???     |       0.1             |       0.9         |       1.0
``` 
이 값은 어떤걸 나타낼까. 

바로 queen 을 나타낸다.

이렇게 embedding 을 하면 단어끼리 연산을 할 수 있다.

문서들을 숫자로 바꿔 벡터에 대한 검색(search)작업을 할 수 있게 된다.




이는 비슷한 벡터를 찾을 수 있게 된다는 것과 같다.

이게 바로 많은 추천 알고리즘들이 작동하는 방식이다.

# 6.4 Vector Store

이전에 split 했던 문서에 embed 작업을 해보자.




In [11]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

# embed 는 문서를 embed 하는 것도 가능하지만, query 도 embed 가능하다.
# embedder.embed_documents()
embedder.embed_query("Hi")

[-0.03629858192333016,
 -0.007224538187570183,
 -0.033718855541097256,
 -0.028663632678071895,
 -0.026865641732513677,
 0.03460482274185761,
 -0.012318847263635711,
 -0.007752209747023989,
 0.0019380524367559973,
 -0.0027018730680822924,
 0.024781013901381176,
 -0.0024771241998875156,
 -0.005732726535614379,
 -0.0029054499465086627,
 0.0066773232887656405,
 -0.0030324821179497563,
 0.03384914384922042,
 -0.0015032120884641694,
 0.021093827586875214,
 -0.008996472123429593,
 -0.02171921630874401,
 0.010384052476961034,
 0.0062441115908914826,
 0.007081220210444346,
 -0.01231233266196503,
 0.0008998100308185957,
 0.005876044512740216,
 -0.009888952994538019,
 -0.0030731974470689,
 -0.024572550373209837,
 0.010742348118267585,
 -0.013810659381252822,
 -0.024429232861745306,
 -0.014110324538845857,
 0.0024347802203507018,
 -0.018878911447619544,
 0.000561872345109932,
 -0.01127001874639878,
 0.018110203351640992,
 -0.009967126351940966,
 0.013028923944578134,
 -0.011328649230112295,
 -0.00

embed 작업을 수행하면 이 단어에 해당하는 벡터를 얻게 된다.

몇개의 차원일까?



In [12]:
len(embedder.embed_query("Hi"))

1536

총 1536 개의 차원을 가지고 있다.

이번에는 문서들을 임베딩 해보자.

이것들은 string 의 배열로 되어 있어야 한다.

In [13]:
vector = embedder.embed_documents(
    [
        "hi",
        "how",
        "are",
        "you",
    ]
)

print(len(vector), len(vector[0]))

4 1536


위 내용을 보면 4개의 벡터가 있고, 각 벡터는 1536 개의 차원을 갖고 있다는 것을 알 수 있다.

이제 실제 우리의 문서를 embed 해보자.

어떻게 하냐면 방금 embed_documents 를 실행하는 것처럼 직접적으로 하지는 않을 것이다.

왜냐하면 코드를 실행할 때마다 문서 embed 를 반복해서 수행하고 싶지 않기 때문이다.

대신, 우리는 그 embed 들을 저장해줄것이다.

랭체인은 embedding 한 것들을 캐싱하는 기능을 제공하고 있다.

문서가 바뀌지 않는 한 embeddings 도 바뀌지 않는다.


vector store 는 일종의 데이터베이스라고 생각할 수 있다.

벡터 공간에서 검색을 할 수 있게 해준다. 여기서는 Chroma 를 사용해보자.

Chroma 에 분할된 문서와 openAI embeddings model 을 전달한다.

In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader(data_path)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(docs, embeddings)

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

작업이 완료되기까지 3초가 걸렸다.

vectorstore에서 유사도 검색을 수행해보자.(vectorstore.similarity_search)

In [15]:
results = vectorstore.similarity_search("where does winston live?")
len(results)

4

In [16]:
results

[Document(page_content='Winston kept his back turned to the telescreen. It was safer, though, as he well knew, even a back can be revealing. A kilometre away the Ministry of Truth, his place of work, towered vast and white above the grimy landscape. This, he thought with a sort of vague distaste -- this was London, chief city of Airstrip One, itself the third most populous of the provinces of Oceania. He tried to squeeze out some childhood memory that should tell him whether London had always been quite like this. Were there always these vistas of rotting nineteenth-century houses, their sides shored up with baulks of timber, their windows patched with cardboard and their roofs with corrugated iron, their crazy garden walls sagging in all directions? And the bombed sites where the plaster dust swirled in the air and the willow-herb straggled over the heaps of rubble; and the places where the bombs had cleared a larger patch and there had sprung up sordid colonies of wooden dwellings li

이제 embeddings 을 캐싱해보자.


In [17]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)


loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

In [18]:
vectorstore

# 6.5 Langsmith

langsmith 와 이것의 설정 방법

이 도구는 우리의 체인이 무엇을 하고있는지 시각적으로 보여준다.

